# Water Quality and Visualizations

## Step 1. Data Cleaning

### Data:

In this project, I will use long-term monitoring data from [Statistics Canada](https://data.ec.gc.ca/data/substances/monitor/national-long-term-water-quality-monitoring-data/). To measure the long-term water quality, Statistics Canada collected data on **145 distinct parameters** from **21 observation sites**.

Out of the 145 distinct water quality parameters, I will investigate **five parameters** in depth. Our parameters of interest are pH, dissolved oxygen, water temperature, dissolved sulphate and dissolved phosphorus. To understand each parameter more in-depth, refer to the table below. 

| **Parameter** | **Description** |
| :- |:------------- | 
| pH | A measure of the hydrogen ion activity in a solution. It indicates how acidic or basic water is. |
|  Water Temperature | Different kinds of aquatic organisms choose to live in varying water temperature. A change in water temperature will lead to changes in types of aquatic life present in water and triggers consecutive chain reactions. |
| Dissolved Oxygen | All aquatic organisms need dissolved oxygen for respiration. Low dissolved oxygen levels can  lead to increased mortality of fish eggs and devastates aquatic ecosystem that are present before. |
| Dissolved Sulphate | Sulphate is the oxidized form of sulphur, which is essential for many biological processes in plants and animals. However, high concentrations of sulphate are detrimental for the survival of aquatic organisms.  |
| Dissolved Phosphorus | Plants and animals need phosphorus to grow. However, high concentrations of phosphorus will enhance vegetation and algae growth, thus breaking the aquatic ecosystem. |

Description from the table was retrieved from [British Columbia government website](https://www2.gov.bc.ca/gov/content/environment/air-land-water/water/water-quality/water-quality-monitoring/canada-bc-water-quality-monitoring-program/water-quality-parameters). 

In [14]:
# import libraries
import pandas as pd
import numpy as np
from scipy import stats

### Data Hindsight

Since the entire dataset will be quite large, let's first look into data from a single location.  

In [2]:
# import data
df = pd.read_csv("https://raw.githubusercontent.com/callysto/data-files/main/data-viz-of-the-week/water-quality/data/okanagan.csv", encoding="latin-1")
print(df.shape)
df.head()

(92418, 11)


,SITE_NO,DATE_TIME_HEURE,FLAG_MARQUEUR,VALUE_VALEUR,SDL_LDE,MDL_LDM,VMV_CODE,UNIT_UNITE,VARIABLE,VARIABLE_FR,STATUS_STATUT
0,BC08NL0001,2000-01-11 09:15,NaN,0.0170,0.0020,NaN,100216,MG/L,ALUMINUM TOTAL,ALUMINIUM TOTAL,P
1,BC08NL0001,2000-01-11 09:15,NaN,0.0003,0.0001,NaN,100250,MG/L,ARSENIC TOTAL,ARSENIC TOTAL,P
2,BC08NL0001,2000-01-11 09:15,NaN,19.6000,0.0100,NaN,100493,UG/L,BARIUM EXTRACTABLE,BARYUM EXTRACTIBLE,P
3,BC08NL0001,2000-01-11 09:15,NaN,0.0192,0.0002,NaN,100217,MG/L,BARIUM TOTAL,BARYUM TOTAL,P
4,BC08NL0001,2000-01-11 09:15,<,0.0020,0.0020,NaN,100474,UG/L,BERYLLIUM EXTRACTABLE,BÉRYLLIUM EXTRACTIBLE,P


Notice that the data has **92418 rows** and **11 columns**. If I compile data from all 21 locations, the resulting dataframe will be massive! Therefore, before proceeding further, I will select specific columns and rows that are of our interest. 

### Data Wrangling:
Now, I proceed to gather data from all locations. Remember that I am only using five parameters, which I need to filter out in the process of data cleaning. Run the following cells to display long-term water quality monitoring data across all provinces in Canada. 

This data may take some time to load, so please wait paitiently.

In [13]:
# Make a list of all locations
df = pd.DataFrame(columns = ["SITE_NO", "LOCATION", "YEAR", "VARIABLE", "VALUE_VALEUR", "UNIT_UNITE"])
observation_locations = ["arctic", "assiniboine", "churchill", "columbia", "fraser",
                         "keewatin", "mackenzie", "missouri", "south_saskatchewan",
                         "north_saskatchewan", "okanagan", "pacific_coastal",
                         "athabasca", "lower_saskatchewan", "winnipeg", "yukon",
                         "gaspe", "maritime", "newfoundland", "st_john", "st_lawrence"]

# Create dictionaries to replace certain entries
prov = {"Newfoundland and Labrador": "NL", "Prince Edward Island":"PE",
       "Nova Scotia":"NS", "New Brunswick":"NB", "Quebec":"QC", "Ontario":"ON",
       "Manitoba":"MB", "Alberta":"AB", "Saskatchewan":"SK", "British Columbia":"BC",
        "Northwest Territories":"NT", "Nunavut":"NU", "Yukon":"YT"}
prov_reverse = {y: x for x, y in prov.items()}

units = {"PH UNITS":"PH",
       "MG/L":"mg/L",
       "DEG C":"°C"}

parameter_rename = {"Phosphorus Total Dissolved":"Dissolved Phosphorus",
                   "Temperature Water":"Water Temperature",
                   "PH":"pH",
                   "Oxygen Dissolved":"Dissolved Oxygen",
                   "Sulphate Dissolved":"Dissolved Sulphate"}

# Gather all observed data
def data_wrangling(lists):
    for alist in lists:
        mydf = pd.read_csv("https://raw.githubusercontent.com/callysto/data-files/main/data-viz-of-the-week/water-quality/data/" + alist + ".csv", encoding='latin-1')
        mydf["LOCATION"] = str(alist)
        mydf["DATE_TIME_HEURE"] = pd.to_datetime(mydf["DATE_TIME_HEURE"])
        mydf["YEAR"] = pd.DatetimeIndex(mydf["DATE_TIME_HEURE"]).year
        mydf = mydf[["SITE_NO", "YEAR", "LOCATION", "VARIABLE", "VALUE_VALEUR", "UNIT_UNITE"]]
        mydf1 = mydf.groupby(["YEAR", "LOCATION", "SITE_NO", "VARIABLE", "UNIT_UNITE"])["VALUE_VALEUR"].mean()
        mydf1 = mydf1.to_frame().reset_index()
        global df
        df = pd.concat([df, mydf1])
    return df

# Add longitude and latitude for each location
sites = pd.read_csv("https://raw.githubusercontent.com/callysto/data-files/main/data-viz-of-the-week/water-quality/data/site_lists.csv", encoding='latin-1')
df = pd.merge(data_wrangling(observation_locations), sites, on="SITE_NO", how="inner")

# Select parameters
parameters = ["PH", "OXYGEN DISSOLVED", "TEMPERATURE WATER", "PHOSPHORUS TOTAL DISSOLVED", "SULPHATE DISSOLVED"]
df = df[df.VARIABLE.isin(parameters)]

# Eliminate out of range values
PH_out_of_range = df[(df['VARIABLE']=='PH') & (~df["VALUE_VALEUR"].between(1,14))].index.values.tolist()
temp_water_out_of_range = df[(df['VARIABLE']=='TEMPERATURE WATER') & (df["VALUE_VALEUR"]<0)].index.values.tolist()
oxygen_out_of_range = df[(df['VARIABLE']=='OXYGEN DISSOLVED') & (df["VALUE_VALEUR"]>20)].index.values.tolist()
all_out_of_range = PH_out_of_range + temp_water_out_of_range + oxygen_out_of_range
df.drop(all_out_of_range, inplace=True)

# Rename columns and further data cleaning 
df = df.rename(columns={"VALUE_VALEUR":"VALUE", "PROV_TERR":"PROVINCE", "UNIT_UNITE":"UNIT"})
df["PROVINCE"] = df["PROVINCE"].replace(prov_reverse)
df["UNIT"] = df["UNIT"].replace(units)
df = df[["PROVINCE", "LOCATION", "YEAR", "SITE_NAME", "SITE_TYPE", "LONGITUDE", "LATITUDE", "VARIABLE", "VALUE", "UNIT"]].reset_index(drop=True)
df["VARIABLE"] = df["VARIABLE"].apply(lambda x: x.title() if (x!="PH") else x)
df["VARIABLE"] = df["VARIABLE"].replace(parameter_rename, regex=True)
df = df.loc[df["YEAR"] < 2020]

# Assign specific colors for each province 
colors=['#1f77b4',  
        '#ff7f0e',  
        '#2ca02c',  
        '#d62728',  
        '#9467bd',  
        '#8c564b', 
        '#e377c2',  
        '#17becf',  
        '#210240',  
        '#21DC49',  
        '#3F5063',  
        '#6C7075',  
        '#F4BC1A'] 
    
color_dict = dict(zip(df["PROVINCE"].unique(), colors))
df["COLOR"] = df["PROVINCE"].map(color_dict)

df.head()

,PROVINCE,LOCATION,YEAR,SITE_NAME,SITE_TYPE,LONGITUDE,LATITUDE,VARIABLE,VALUE,UNIT,COLOR
0,Northwest Territories,arctic,2000,HORNADAY RIVER BELOW UNNAMED EASTERN TRIBUTARY,RIVER/RIVIÈRE,-122.4022,68.7542,pH,7.9825,PH,#1f77b4
1,Northwest Territories,arctic,2000,HORNADAY RIVER BELOW UNNAMED EASTERN TRIBUTARY,RIVER/RIVIÈRE,-122.4022,68.7542,Dissolved Phosphorus,0.007,mg/L,#1f77b4
2,Northwest Territories,arctic,2000,HORNADAY RIVER BELOW UNNAMED EASTERN TRIBUTARY,RIVER/RIVIÈRE,-122.4022,68.7542,Dissolved Sulphate,10.05,mg/L,#1f77b4
3,Northwest Territories,arctic,2000,HORNADAY RIVER BELOW UNNAMED EASTERN TRIBUTARY,RIVER/RIVIÈRE,-122.4022,68.7542,Water Temperature,6.95,°C,#1f77b4
4,Northwest Territories,arctic,2002,HORNADAY RIVER BELOW UNNAMED EASTERN TRIBUTARY,RIVER/RIVIÈRE,-122.4022,68.7542,pH,7.855,PH,#1f77b4


I can also look into the **provincial average** of the parameter concentrations (values). The dataframe below will display a summarized form of the previous dataframe, with the provincial average values for each year, for the five selected parameters. 

In [4]:
# Provincial summary of yearly average parameter concentrations
df_summary = df.groupby(["PROVINCE", "VARIABLE", "YEAR", "UNIT"])["VALUE"].mean().to_frame()
df_summary = df_summary.reset_index()
print(df_summary.shape)
df_summary.head()

(1024, 5)


,PROVINCE,VARIABLE,YEAR,UNIT,VALUE
0,Alberta,Dissolved Oxygen,2000,mg/L,10.402069
1,Alberta,Dissolved Oxygen,2001,mg/L,10.925248
2,Alberta,Dissolved Oxygen,2002,mg/L,10.539742
3,Alberta,Dissolved Oxygen,2003,mg/L,10.522747
4,Alberta,Dissolved Oxygen,2004,mg/L,10.126394


The summarized dataframe consists of **1024 rows** and **5 columns**. I will use the summarized dataframe to create a line graph and the original dataframe to create a series of data-plotted maps. From now, I will refer to the summarized dataframe as `df_summary`, and the original dataframe as `df`.